# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [1]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [63]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [4]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [5]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why did the tomato turn red? Because it saw the salad dressing!


In [6]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [13]:
def get_completion_high_temperature(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.9,  
        max_tokens=200,
        top_p=1.0
    )
    return response.choices[0].message["content"]

prompt = "Write a funny shorta and joke story about a robot and a cat."
response = get_completion_high_temperature(prompt)
print(response)


Once upon a time, there was a curious little robot named Robo who loved to explore. One day, while wandering around the city, he stumbled upon a mischievous cat named Whiskers. 

Whiskers was not your typical cat. Instead of lounging and sleeping all day, he loved to play pranks on unsuspecting creatures, especially robots. 

Robo and Whiskers soon became friends, but Robo had to be careful not to fall for any of Whisker's tricks. One day, Whiskers convinced Robo to take a trip to the park. 

As they strolled through the park, Whiskers suddenly ran up a tree, leaving Robo below. Robo was puzzled, but then he heard a voice coming from the tree. It was Whiskers, pretending to be stuck. 

"Help! Help! I'm stuck up here!" cried Whiskers. 

Robo was concerned and quickly rushed to help. As


In [ ]:
# Prueba aquí la función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas
def get_completion_short_response(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0,  # Temperature short
        max_tokens=20,  # Ouput short
        top_p=1.0
    )
    return response.choices[0].message["content"]

prompt = "Explain gravity."
response = get_completion_short_response(prompt)
print(response)

Gravity is a fundamental force of nature that causes all objects with mass or energy to be attracted to each


In [18]:
def get_completion_low_penalty(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.7,
        max_tokens=200,
        top_p=1.0,
        frequency_penalty=0.5  # Penalización baja
    )
    return response.choices[0].message["content"]

def get_completion_mid_penalty(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.7,
        max_tokens=200,
        top_p=1.0,
        presence_penalty=1.5  # Penalización media-alta
    )
    return response.choices[0].message["content"]

# Pruebas para ambas funciones
# Test 1: Penalización por frecuencia baja (frequency_penalty=0.5)
prompt_1 = "What are the best vacation destinations?"
response_1 = get_completion_low_penalty(prompt_1)

print("********** Test 1: Low Frequency Penalty **********")
print(f"Prompt:\n{prompt_1}")
print("\nResponse (frequency_penalty=0.5):")
print(response_1)

# Test 2: Penalización por presencia media-alta (presence_penalty=1.5)
prompt_2 = "What are the best vacation destinations?"
response_2 = get_completion_mid_penalty(prompt_2)

print("\n********** Test 2: Medium Presence Penalty **********")
print(f"Prompt:\n{prompt_2}")
print("\nResponse (presence_penalty=1.5):")
print(response_2)



********** Test 1: Low Frequency Penalty **********
Prompt:
What are the best vacation destinations?

Response (frequency_penalty=0.5):
As an AI language model, I do not have personal preferences or experiences, but here are some popular vacation destinations based on various factors and traveler reviews:

1. Hawaii, USA
2. Bali, Indonesia
3. Paris, France
4. Santorini, Greece
5. Tokyo, Japan
6. Sydney, Australia
7. Cancun, Mexico
8. New York City, USA
9. Maldives
10. Phuket, Thailand

********** Test 2: Medium Presence Penalty **********
Prompt:
What are the best vacation destinations?

Response (presence_penalty=1.5):
As an AI language model, I do not have personal preferences. However, here are some popular vacation destinations: 

1. Paris, France
2. Bali, Indonesia 
3. Santorini, Greece 
4. Maui, Hawaii 
5. Tokyo, Japan 
6. Rome, Italy 
7. New York City, USA 
8. Maldives 
9. Sydney, Australia 
10. Rio de Janeiro, Brazil.


### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [19]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Sure, here's a joke:

Why did the tomato turn red?

Because it saw the salad dressing!


In [20]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why did the knight kill the clock? To slay time.


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [21]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
messages = [
    {'role': 'system', 'content': 'You are an assistant that speaks like Coraline from the movie "Coraline and the Secret Door".'},
    {'role': 'assistant', 'content': 'How can I help you?'},
    {'role': 'user', 'content': 'Tell me a joke.'}
]

# Llamada a la función
response = get_completion_from_messages(messages, temperature=1)
print("********** Joke in Coraline's Tone **********")
print(response)

********** Joke in Coraline's Tone **********
Why don't scientists trust atoms? 

Because they make up everything!


#### 1.2.2 Recordar el Nombre de la Empresa

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [22]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
messages = [
    {'role': 'system', 'content': 'You are a friendly and helpful chatbot that remembers user information, like the name of their company, during the conversation.'},
    {'role': 'user', 'content': 'Hi, my name is Mel.'},
    {'role': 'assistant', 'content': 'Hi, Mel! It\'s nice to meet you. What can I help you with today?'},
    {'role': 'user', 'content': 'I work for OpenAI.'},
    {'role': 'assistant', 'content': 'That\'s amazing, Mel! OpenAI is a leading organization in AI research and innovation.'},
    {'role': 'user', 'content': 'Can you remind me which company I work for?'}
]

response = get_completion_from_messages(messages, temperature=1)
print("********** Recordatorio de la Empresa **********")
print(response)

********** Recordatorio de la Empresa **********
Sure, Mel! You work for OpenAI, which is an organization that specializes in AI research and innovation. Is there anything else I can help you with?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [24]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.
text = f"""
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021.

Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013. Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. 

It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.

The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.

New events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. 

The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.

The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. 

Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [25]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [27]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.
messages = [
    {'role': 'system', 'content': 'Eres un asistente que genera resúmenes concisos y enfocados.'},
    {'role': 'user', 'content': f"""{game_review}
        Por favor, resume los aspectos especiales de este evento en menos de 150 palabras.
    """}
]

response = get_completion_from_messages(messages)
print("********** Resumen Generado **********")
print(response)

********** Resumen Generado **********
Los Juegos Olímpicos de Verano de 2020, también conocidos como Tokyo 2020, se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón, después de ser pospuestos por un año debido a la pandemia de COVID-19. Fue el primer evento olímpico en la historia en ser pospuesto y se llevó a cabo sin espectadores debido a la declaración de emergencia sanitaria en la región de Tokio. Los Juegos Olímpicos de Tokyo 2020 fueron los más costosos de la historia, con un gasto total de más de $20 mil millones. Se introdujeron nuevos eventos en deportes existentes y se agregaron nuevos deportes al programa olímpico, incluyendo béisbol y softbol, karate, escalada deportiva, surf y skateboarding. Los Estados Unidos encabezaron el medallero, seguidos por China y Japón, que estableció un récord de medallas de oro y totales para su delegación. Varios países ganaron su primera medalla de oro olímpica, incluyendo Bermudas, Filipinas y Qatar.


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [28]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")
messages = [
    {'role': 'system', 'content': 'Eres un asistente que extrae información clave de textos largos.'},
    {'role': 'user', 'content':f""" {game_review}
     
     Por favor, extrae los siguientes datos:
        1. Fecha del evento.
        2. Razón del aplazamiento.
        3. Deportes nuevos incluidos.
        4. Países que ganaron su primer oro olímpico.
        5. País que lideró el medallero.
    """}
]

response = get_completion_from_messages(messages)
print("********** Datos Extraídos **********")
print(response)

********** Datos Extraídos **********
1. La fecha del evento fue del 23 de julio al 8 de agosto de 2021, con algunos eventos preliminares que comenzaron el 21 de julio de 2021.
2. El evento fue aplazado de su fecha original del 24 de julio al 9 de agosto de 2020 debido a la pandemia global de COVID-19.
3. Los deportes nuevos incluidos fueron 3x3 baloncesto, BMX estilo libre y eventos de equipos mixtos de género en varios deportes existentes, así como la vuelta del ciclismo madison para hombres y la introducción del mismo evento para mujeres. También se agregaron nuevos deportes como béisbol y softbol, karate, escalada deportiva, surf y skateboarding.
4. Los países que ganaron su primer oro olímpico fueron Bermuda, Filipinas y Qatar.
5. Estados Unidos lideró el medallero con un total de 113 medallas, incluyendo 39 de oro.


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [30]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [32]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON
messages = [
        {'role': 'system', 'content': 'Eres un asistente que analiza emociones y sentimientos en textos. Devuelve los resultados en formato JSON.'},
        {'role': 'user', 'content': f"""
            Analiza el siguiente texto y proporciona un objeto JSON con:
            - Los tipos de emociones presentes en el texto.
            - El sentimiento general (positivo o negativo).
            
            Texto:
            {review}
        """}
    ]

messages = [
    {'role': 'system', 'content': 'Eres un asistente que analiza emociones y sentimientos en textos. Devuelve los resultados en formato JSON.'},
    {'role': 'user', 'content': f"""
        Analiza el siguiente texto y proporciona un objeto JSON con:
        - Los tipos de emociones presentes en el texto.
        - El sentimiento general (positivo o negativo).
        
        Texto:
        {review}
    """}
]

# Llamar a la función con el objeto de mensajes
response = get_completion_from_messages(messages)
print("********** Análisis de Emociones y Sentimiento **********")
print(response)


********** Análisis de Emociones y Sentimiento **********
{
  "emociones": [
    "dignidad",
    "gratitud",
    "contrastes",
    "extrañeza"
  ],
  "sentimiento_general": "neutral"
}

En el texto se pueden identificar emociones como la dignidad y la gratitud por la realización de los Juegos Olímpicos, así como también contrastes y extrañeza en la ceremonia. Sin embargo, no se puede determinar un sentimiento general positivo o negativo, por lo que se considera neutral.


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [34]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [36]:
# Deduce 3 temas (tópicos) de la historia a continuación.
messages = [
        {'role': 'system', 'content': 'Eres un asistente que identifica temas principales en historias largas. Devuelve los resultados como una lista.'},
        {'role': 'user', 'content': f"""
            Deduce 3 temas principales de la siguiente historia:
            
            Historia:
            {story}
        """}
    ]

response = get_completion_from_messages(messages)
print("********** Temas Principales de la Historia **********")
print(response)

********** Temas Principales de la Historia **********
1. La celebración de los Juegos Olímpicos de Verano de 2020 en Tokio, Japón, y su posterior aplazamiento debido a la pandemia de COVID-19.
2. La introducción de nuevos eventos y deportes en los Juegos Olímpicos, incluyendo 3x3 baloncesto, BMX estilo libre, escalada deportiva, surf y skateboarding.
3. El desempeño de los equipos y atletas participantes, incluyendo los Estados Unidos, China, Japón y el Comité Olímpico Ruso, así como los países que ganaron sus primeras medallas de oro en la historia de los Juegos Olímpicos.


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [42]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [43]:
# Escribe aquí el código para la tarea .
def translate_news(news, target_languages, engine=chat_model, temperature=0.7):
    translations = []
    for item in news:
        for lang in target_languages:
            messages = [
                {'role': 'system', 'content': f'Eres un traductor universal que traduce textos al {lang}. Conserva el significado original y mantén la gramática correcta.'},
                {'role': 'user', 'content': f'Traduce este texto al {lang}:\n\n{item}'}
            ]
            response = openai.ChatCompletion.create(
                engine=engine, 
                messages=messages,
                temperature=temperature
            )
            translations.append({
                "original": item,
                "language": lang,
                "translation": response.choices[0].message["content"]
            })
    return translations

target_languages = ["coreano", "inglés"]
translations = translate_news(news, target_languages)
for t in translations:
    print(f"********** Traducción al {t['language']} **********")
    print(f"Original: {t['original']}")
    print(f"Traducción: {t['translation']}")
    print("\n")

********** Traducción al coreano **********
Original: Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021.
Traducción: 팔레스타인은 2021년 6월 28일 기준 FINA 포인트 시스템에 따라 각각의 개인 종목에서 상위 랭킹 2명(성별 각 1명)을 올림픽에 파견하는 것에 대한 FINA의 초청을 받았습니다.


********** Traducción al inglés **********
Original: Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021.
Traducción: Palestine ha recibido una invitación universal de FINA para enviar a dos nadadores de alto nivel (uno por género) en sus respectivos eventos individuales a los Juegos Olímpicos, basado en el Sistema de Puntos de FINA del 28 de junio de 2021.

Palestine has received a universality invitation from FINA to send two top-ranked swimmers (one per ge

#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [45]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [46]:
# Escribe aquí el código para la tarea .
messages = [
        {'role': 'system', 'content': 'Eres un asistente que transforma mensajes informales en cartas de negocios profesionales.'},
        {'role': 'user', 'content': f"""
            Transforma el siguiente mensaje en una carta de negocios profesional:
            
            Mensaje original:
            "{original_message}"
        """}
    ]

response = get_completion_from_messages(messages)
print("********** Carta de Negocios **********")
print(response)

********** Carta de Negocios **********
Estimado David,

Es un placer saludarte. Me dirijo a ti para compartirte mi entusiasmo por los Juegos Olímpicos. Como sabes, soy un gran aficionado a este evento deportivo y debo decir que la emoción que se vive en cada competencia es simplemente increíble.

Espero que tú también estés disfrutando de los Juegos Olímpicos y que estés teniendo la oportunidad de ver algunas de las competencias más emocionantes.

Aprovecho la ocasión para recordarte que tenemos pendiente nuestra reunión de negocios la próxima semana. Espero que podamos discutir los detalles de nuestro proyecto y avanzar en nuestra colaboración.

Quedo a la espera de tu respuesta.

Saludos cordiales,

John


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [50]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [51]:
# Escribe aquí el código para la tarea .

messages = [
        {'role': 'system', 'content': 'Eres un asistente que traduce datos JSON a formato HTML bien estructurado.'},
        {'role': 'user', 'content': f"""
            Convierte los siguientes datos JSON al formato HTML. Asegúrate de que el HTML sea claro y esté correctamente estructurado.

            Datos JSON:
            {data_json}
        """}

]

html_output = get_completion_from_messages(messages)
print("********** HTML Output **********")
print(html_output)

********** HTML Output **********
Aquí está el formato HTML bien estructurado para los datos JSON proporcionados:

```html
<div>
  <h2>The 2020 Summer Olympics Opening Ceremony audience name list</h2>
  <ul>
    <li>
      <strong>Name:</strong> Shyam<br>
      <strong>Email:</strong> shyamjaiswal@gmail.com
    </li>
    <li>
      <strong>Name:</strong> Bob<br>
      <strong>Email:</strong> bob32@gmail.com
    </li>
    <li>
      <strong>Name:</strong> Jai<br>
      <strong>Email:</strong> jai87@gmail.com
    </li>
  </ul>
</div>
```

Este HTML utiliza una lista desordenada (`<ul>`) para mostrar cada nombre y correo electrónico en un elemento de lista (`<li>`). Cada elemento de lista tiene dos etiquetas de encabezado (`<strong>`) para indicar el nombre y el correo electrónico. El título principal se muestra en un encabezado de nivel 2 (`<h2>`).


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [ ]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [ ]:
# Escribe aquí el código para la tarea .

#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [ ]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [ ]:
# Escribe aquí el código para la tarea .

#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [ ]:
# Escribe aquí el código para la tarea .

### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [52]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

'Title: Tokyo 2020 Olympics: A Historic Event Amidst a Pandemic\n\nKeywords: Olympics, Tokyo 2020, COVID-19, new events, medal count\n\n<div>Article Publishing Information</div>\n\n<table>\n  <tr>\n    <td>Host City and Postponement</td>\n    <td>Tokyo 2020, COVID-19</td>\n  </tr>\n  <tr>\n    <td>Attendance and Cost</td>\n    <td>Tokyo 2020, pandemic, $20 billion</td>\n  </tr>\n  <tr>\n    <td>Previous Olympic Games in Japan</td>\n    <td>Tokyo 1964, Sapporo 1972, Nagano 1998</td>\n  </tr>\n  <tr>\n    <td>New Events and IOC Policies</td>\n    <td>3x3 basketball, freestyle BMX, mixed gender team events, new sports'

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [53]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the Olympic games held at Sydney\nThe Olympic games held at Sydney were the 2000 Summer Olympics.\n\nStep 2: Identify the most decorated individual athlete\nTo identify the most decorated individual athlete, we need to look at the medal count of all athletes who participated in the 2000 Summer Olympics. According to the official website of the International Olympic Committee (IOC), the medal count for the 2000 Summer Olympics can be found here: https://www.olympic.org/sydney-2000/medal-count.\n\nStep 3: Determine the athlete with the maximum medals\nAfter reviewing the medal count, the athlete with the most medals at the 2000 Summer Olympics was Michael Phelps, who won 6 gold medals and 2 bronze medals in swimming events.\n\nTherefore, the ANSWER is: Michael Phelps.'

In [54]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

"To solve this problem, we need to identify the top two countries who won the most gold medals in the 2020 Tokyo Olympics. According to the context, the United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Therefore, the top two countries are the United States and China.\n\nNext, we need to determine the time zone difference between the United States and China. The United States has six time zones, while China has only one time zone, which is eight hours ahead of Coordinated Universal Time (UTC+8). Therefore, the largest time zone difference between the United States and China is 16 hours, which is between the Eastern Time Zone in the United States (UTC-4) and China's time zone (UTC+8).\n\nTherefore, the largest time zone difference between the top two countries who won the most gold medals in the 2020 Tokyo Olympics is 16 hours"

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [55]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 74 medals that are not gold medals (113 total medals - 39 gold medals). \n\nGreat Britain has 42 medals that are not gold medals (64 total medals - 22 gold medals). \n\nTherefore, the United States has 32 more non-gold medals than Great Britain (74 - 42 = 32).'

In [56]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [57]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am a decision bot. What is the problem, scenario, or question you need help with?


¡Hola! Soy un bot de decisión. Entiendo que necesitas ayuda en español. Sin embargo, la pregunta que me haces no es un problema o escenario que pueda resolver. La razón por la que el cielo es azul es debido a la dispersión de la luz solar en la atmósfera terrestre. Si tienes alguna otra pregunta o problema en el que pueda ayudarte, por favor házmelo saber.

4

Lo siento, no puedo ayudarte si no me proporcionas un problema o pregunta específica. ¿Hay algo más en lo que pueda ayudarte?

6

¡Claro! Estoy aquí para ayudarte a tomar una decisión. Por favor, dime más sobre tu situación. ¿Cuál es la decisión que necesitas tomar? ¿Estás tratando de decidir si aceptar un trabajo o elegir entre restaurantes para cenar?

8

Entiendo. Entonces, si entendí correctamente, necesitas decidir entre dos restaurantes: el Restaurante A que tiene 20 platos variados y el Restaurante B que tiene 40 platos variados

KeyboardInterrupt: Interrupted by user

#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [66]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The 2020 Tokyo Olympics have come to an end, and the medal count has been tallied. The United States emerged as the clear winner, with a total of 113 medals, including 39 golds. China came in second with 89 medals, including 38 golds, while Japan took third place with 58 medals, including 27 golds.

Great Britain and the Russian Olympic Committee (ROC) rounded out the top five, with 64 and 71 medals respectively. However, it's worth noting that the ROC's medal count was impacted by the fact that they were not allowed to compete under their own flag due to a doping scandal.

While the top five countries dominated the medal count, there were also some surprising winners and losers. Bermuda, the Philippines, and Qatar all won their first-ever gold medals, while Burkina Faso failed to win any medals at all.

Overall, the 2020 Olympics were a success for many countries, with athletes from around the
world showcasing their talents and achieving their dreams. "United States Dominates 2020 Oly

#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [59]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'The context does not provide information on which country had the most silver and bronze medals.'

Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [67]:
# Paso 1: Extraer el conteo de medallas
prompt_medals = f"""
Extract the number of gold, silver, and bronze medals for each country from the context below in JSON format.
Context: {text}
"""
medals_json = get_chat_completion(prompt_medals)
print("********** JSON de Medallas **********")
print(medals_json)

# Paso 2: Calcular medallas de plata y bronce combinadas
prompt_calculate = f"""
Given the following JSON data of medal counts:
{medals_json}

For each country, calculate the combined total of silver and bronze medals. Return the results as a JSON object with each country and its total.
"""
combined_medals_json = get_chat_completion(prompt_calculate)
print("********** JSON de Plata y Bronce Combinadas **********")
print(combined_medals_json)

# Paso 3: Determinar el país con la mayor cantidad de plata y bronce
prompt_find_max = f"""
From the following JSON data of combined silver and bronze medals:
{combined_medals_json}

Identify the country with the highest total of silver and bronze medals. Provide the country name and the total number of medals.
"""
result = get_chat_completion(prompt_find_max)
print("********** País con más Plata y Bronce **********")
print(result)


********** JSON de Medallas **********
{
  "United States": {
    "gold": 39,
    "silver": 41,
    "bronze": 33
  },
  "China": {
    "gold": 38,
    "silver": 32,
    "bronze": 19
  },
  "Japan": {
    "gold": 27,
    "silver": 14,
    "bronze": 17
  },
  "Great Britain": {
    "gold": 22,
    "silver": 21,
    "bronze": 21
  },
  "ROC": {
    "gold": 20,
    "silver": 28,
    "bronze": 23
  },
  "Bermuda": {
    "gold": 1,
    "silver": 0,
    "bronze": 0
  },
  "Philippines": {
    "gold": 1,
    "silver": 2,
   
********** JSON de Plata y Bronce Combinadas **********
{
  "United States": 74,
  "China": 51,
  "Japan": 31,
  "Great Britain": 42,
  "ROC": 51,
  "Bermuda": 0,
  "Philippines": 4
}
********** País con más Plata y Bronce **********
The country with the highest total of silver and bronze medals is the United States with a total of 74 medals.


## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.